In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

In [ ]:
#import data from csv
dataVR1 = pd.read_excel("\path\to\folder\",sheet_name="Raw")
dataSeg1 = pd.read_excel("\path\to\folder\")

In [ ]:
# print('---VR---')
# print(dataVR1.head(5))
# print('---Seg---')
# print(dataSeg1.head(5))

In [ ]:
#clean imported data
#--- VR Data ---
selectVR = [0,1,2,4,5,6,7]
dataVR = dataVR1.iloc[:,selectVR]
columsVR = ['period','product_long','GRID_EUP','revenue','product_short','product_group','TS_client']
dataVR.columns = columsVR
dataVR = dataVR[dataVR['period']==201712]
#--- Seg Data ---
selectSeg = [0,1,2,3,5,7,8,9]
dataSeg = dataSeg1.iloc[:,selectSeg]
columsSeg = ['segment','clientName','GRID_EUP','pam','pamLoc','sector','region','valueProp']
dataSeg.columns = columsSeg

In [ ]:
print('---VR---')
print(dataVR.head(5))
print('---Seg---')
print(dataSeg.head(5))

In [ ]:
# Create merged dataframe
fullData = pd.merge(dataVR, dataSeg, how="inner", on="GRID_EUP")


In [ ]:
# Graph 1 - TS Potential mapping
#graph1Data = fullData.groupby(['segment','TS_client'],as_index=False)[['revenue']].count()
YesTS = fullData[fullData['product_group'] == 'TS']
YesTSEUP = YesTS['GRID_EUP'].unique()

fullData1 = fullData.assign(TSclient=2)

runEnd = fullData1.shape[0]

for i in range(runEnd) :
    if fullData1.iloc[i,2] in YesTSEUP :
        fullData1.loc[i,'TSclient'] = 1
    else :
        fullData1.loc[i,'TSclient'] = 0

In [ ]:
graph1Data = fullData1.groupby(by=['segment','TSclient'], as_index=False).agg({'GRID_EUP': pd.Series.nunique})
print(graph1Data)

graph1Data.to_excel('graph1Data.xlsx')
#fullData1.to_excel('fullData1.xlsx')

In [ ]:
# Graph 2 - Revenue section Potential mapping
tsFullData = fullData1[fullData1['TSclient'] == 1]
tsFullData = tsFullData.groupby(['GRID_EUP','clientName','segment','TS_client','TSclient'],as_index=False)[['revenue']].sum()

tsFullRev = tsFullData.groupby(['GRID_EUP'],as_index=False)[['revenue']].sum()
tsFullData1 = pd.merge(tsFullData, tsFullRev, how="inner", on="GRID_EUP")
tsFullData1['revShareTS'] = tsFullData1['revenue_x'] / (tsFullData1['revenue_y']+1)

tsRevShare = tsFullData1[tsFullData1['TS_client'] == 'yes']
tsRevShareTop = tsRevShare[tsRevShare['revShareTS'] >= 0.5]
tsRevShareMid = tsRevShare[np.logical_and(tsRevShare['revShareTS'] >= 0.25,tsRevShare['revShareTS'] < 0.50)]
tsRevShareLow = tsRevShare[tsRevShare['revShareTS'] < 0.25]

#Result
allEUP = fullData1['GRID_EUP'].unique()
tsEUP = tsFullData['GRID_EUP'].unique()

TsTopEUP = tsRevShareTop['GRID_EUP'].unique()
TsMidEUP = tsRevShareMid['GRID_EUP'].unique()
TsLowEUP = tsRevShareLow['GRID_EUP'].unique()

ShareTS = float(tsEUP.size)/float(allEUP.size)
ShareTop = float(TsTopEUP.size)/float(tsEUP.size)
ShareMid = float(TsMidEUP.size)/float(tsEUP.size)
ShareLow = float(TsLowEUP.size)/float(tsEUP.size)


print(ShareTS)
print'---split---'
print(ShareTop)
print(ShareMid)
print(ShareLow)
print(ShareTop+ShareMid+ShareLow)
print('---Num---')
print(float(allEUP.shape[0]))
print(float(tsEUP.shape[0]))
print(float(TsTopEUP.size))
print(float(TsMidEUP.size))
print(float(TsLowEUP.size))
print(float(TsTopEUP.size)+float(TsMidEUP.size)+float(TsLowEUP.size))


tsRevShare.to_excel('tsRevShare.xlsx')

In [ ]:
tsRevShare = tsRevShare.sort_values(by='revenue_x', ascending=False)
tsRevShare['runningTotal'] = 1
tsRevShare.iloc[0,8] = tsRevShare.iloc[0,6]


end = tsRevShare.shape[0]

for j in range(1,end) :
    k = j-1
    tsRevShare.iloc[j,8] = tsRevShare.iloc[j,6] + tsRevShare.iloc[j-1,8]


In [ ]:
#end-1
tsRevShare['totRevShareTS'] = 0.000000
tsRevShare['totRevShareTS'] = tsRevShare.iloc[:,8]/tsRevShare.iloc[-1,8]
topRevCust = tsRevShare[tsRevShare['totRevShareTS']<0.8]
topRevCust = topRevCust.sort_values(by='totRevShareTS', ascending=True)

YesTSTop = topRevCust['GRID_EUP'].unique()

fullData3 = fullData.assign(TSTopclient=0)

runEnd3 = fullData3.shape[0]

for i in range(runEnd3) :
    if fullData3.iloc[i,2] in YesTSTop :
        fullData3.loc[i,'TSTopclient'] = 1
    
#tsRevShare.iloc[4680,8] = tsRevShare.iloc[4680,6] + tsRevShare.iloc[4679,8]
print(topRevCust.head(5))

In [ ]:
graph2Data = fullData3.groupby(by=['segment','product_group'], as_index=False).agg({'revenue': pd.Series.sum})
graph3Data = fullData3.groupby(by=['segment','product_short'], as_index=False).agg({'revenue': pd.Series.sum})
print(graph2Data)

graph2Data.to_excel('graph2_2Data.xlsx')
graph3Data.to_excel('graph2_3Data.xlsx')